In [1]:
import os
import random
import imagehash 
import numpy as np 
import pandas as pd
from PIL import Image 
from annoy import AnnoyIndex 
import glob

In [2]:
data_dir = '/gpfs01/berens/data/data/eyepacs/data_processed/images/'
csv_file = '/gpfs01/berens/data/data/eyepacs/data_processed_clahe_resized/metadata/splits_longitudinal_test/qual_eval_train.csv'
df = pd.read_csv(csv_file, low_memory=False)
filenames = df['image_path']
filenames = filenames.apply(lambda x: os.path.join(data_dir, x))
filenames = filenames.tolist()

samples_dir = '/gpfs01/berens/user/iilanchezian/Projects/guided-diffusion/diffusion_model_samples_no_rotate_5_class_balancing_only_train/'
samples = os.listdir(samples_dir)
samples_path = list(map(lambda x: os.path.join(samples_dir, x), samples))
filenames.extend(samples_path)

print(len(filenames))

47924


In [3]:
vector_length = 0
sz = (256,256)
id_to_vec = {}
for count,f in enumerate(filenames):
    img = Image.open(f)
    img = img.resize(sz)
    img_hash = imagehash.whash(img)
    hash_array = img_hash.hash.astype('int').flatten();
    vector_length = hash_array.shape[0]
    id_to_vec[count] = hash_array

UnidentifiedImageError: cannot identify image file '/gpfs01/berens/user/iilanchezian/Projects/guided-diffusion/diffusion_model_samples_no_rotate_5_class_balancing_only_train/samples_1000x256x256x3.npz'

In [ ]:
f = vector_length
dist_function = "hamming"

t = AnnoyIndex(f, dist_function)
for key,value in id_to_vec.items():
    t.add_item(key,value)

In [ ]:
num_trees = 200
t.build(num_trees)

In [ ]:
test_file = '/gpfs01/berens/user/iilanchezian/Projects/guided-diffusion/'
query_index = filenames.index(test_file)
num_neighbors = 9
neighbors = t.get_nns_by_item(query_index,num_neighbors,include_distances=True)